[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/betloreilly/hybridsearch_llmevaluation/blob/main/Hybrid%20Search%20and%20LLM%20Evaluation.ipynb)

## Astra DB's Hybrid Search and LLM Evaluation

The goal of this tutorial is to demonstrate how to build a system that can efficiently identify and evaluate the top job opportunities for candidates.

We will build together a sample end-to-end use case that will be able to find and evaluate the best job opportunities for candidates by using the power of LLM and Astra Hybrid Search capabilities.


## Install Dependencies

In [1]:
!pip install python-dotenv langchain openai sentence-transformers cassio tiktoken python-dotenv
!pip install cassandra-driver

17.31s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


DEBUG: BASH_SOURCE[0] -- 
/Users/madhavansridharan/Data/Tools/cloud-ondemand/support-tools/kube/bashrc.sh:source:5: no such file or directory: /Users/madhavansridharan/Data/Tools/sperf_and_nibbler/hybridsearch_llmevaluation/../../sso_functions.sh
/Users/madhavansridharan/Data/Tools/cloud-ondemand/support-tools/kube/bashrc.sh:7: command not found: is_aws_sso_enabled
/Users/madhavansridharan/Data/Tools/cloud-ondemand/support-tools/kube/bashrc.sh:source:12: no such file or directory: /Users/madhavansridharan/Data/Tools/sperf_and_nibbler/hybridsearch_llmevaluation/commonrc.sh
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.9/124.9 KB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 KB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
ERROR: Could not find a version that satisfies the requirement cassio (from versions: none)
ERROR: No matching distribution found for cassi

25.26s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


DEBUG: BASH_SOURCE[0] -- 
/Users/madhavansridharan/Data/Tools/cloud-ondemand/support-tools/kube/bashrc.sh:source:5: no such file or directory: /Users/madhavansridharan/Data/Tools/sperf_and_nibbler/hybridsearch_llmevaluation/../../sso_functions.sh
/Users/madhavansridharan/Data/Tools/cloud-ondemand/support-tools/kube/bashrc.sh:7: command not found: is_aws_sso_enabled
/Users/madhavansridharan/Data/Tools/cloud-ondemand/support-tools/kube/bashrc.sh:source:12: no such file or directory: /Users/madhavansridharan/Data/Tools/sperf_and_nibbler/hybridsearch_llmevaluation/commonrc.sh
  Using cached cassandra_driver-3.28.0-cp37-cp37m-macosx_10_9_x86_64.whl (4.0 MB)
  Using cached geomet-0.2.1.post1-py3-none-any.whl (18 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 KB 2.8 MB/s eta 0:00:00
You should consider upgrading via the '/Users/madhavansridharan/.pyenv/versions/3.7.14/bin/python3.7 -m pip install --upgrade pip' command.


## Importing neccessary libraries and organizing configuration

<!-- Here you can find the `conf.env` template:
>   ```
>   ASTRA_API_KEY = 
>   ASTRA_SECURE_CONNECT_BUNDLE_PATH = 
>   OPENAI_API_KEY=  
>   ``` -->


In [3]:
# Config
import os
import pandas as pd
import numpy as np
import json
from getpass import getpass

from cassandra.cluster import Cluster, ExecutionProfile, EXEC_PROFILE_DEFAULT, ProtocolVersion
from cassandra.auth import PlainTextAuthProvider
from langchain.vectorstores import Cassandra
from langchain.schema.document import Document

from langchain.embeddings import OpenAIEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings, SentenceTransformerEmbeddings

profile = ExecutionProfile(request_timeout=30)

# Set parameters for Astra DB connection
os.environ["ASTRA_API_KEY"] = getpass('Enter your Astra DB API Key/Token ("AstraCS:..."): ')
os.environ["ASTRA_SECURE_CONNECT_BUNDLE_PATH"] = ''
os.environ["ASTRA_DB_KEYSPACE"] = input('Enter your Astra DB Keyspace name (e.g. vector): ')
os.environ["ASTRA_DB_TABLE_NAME"] = input('Enter your Astra DB Table name (e.g. jobs): ')
os.environ["OPENAI_API_KEY"] = getpass('Enter your OpenAI API key: ')

#assert os.environ.get("ASTRA_DB_KEYSPACE"), "⚠️ Please provide Astra DB keyspace name"
#assert os.environ.get("ASTRA_DB_TABLE_NAME"), "⚠️ Please provide Astra DB table name"


ModuleNotFoundError: No module named 'pandas'

In [ ]:
# Upload your Astra DB secure connect bundle
import os
from google.colab import files

print('Please upload your Astra DB Secure Connect Bundle')
uploaded = files.upload()
if uploaded:
    astraBundleFileTitle = list(uploaded.keys())[0]
    ASTRA_DB_SECURE_BUNDLE_PATH = os.path.join(os.getcwd(), astraBundleFileTitle)
else:
    raise ValueError(
        'Cannot proceed without Astra DB\'s Secure Connect Bundle. Please re-run the cell.'
    )

## Prepare table schema

Here is the DDL commands to get our schema up and running for later use.
You can find the table and index creation scripts below:
```
CREATE TABLE vector.jobs (
  job_id text PRIMARY KEY,
  job_title text,
  skills text,
  salary text,
  location text,
  embedding_vector vector<float, 1536> )

CREATE CUSTOM INDEX IF NOT EXISTS ann_index
  ON vector.jobs(embedding_vector) USING 'StorageAttachedIndex';

CREATE CUSTOM INDEX ix_location ON vector.jobs(Location ) USING 'org.apache.cassandra.index.sai.StorageAttachedIndex' WITH OPTIONS = {
'index_analyzer': '{
        "tokenizer" : {"name" : "standard"},
        "filters" : [{"name" : "porterstem"}]
}'};


CREATE CUSTOM INDEX ix_salary ON vector.jobs(salary) USING 'org.apache.cassandra.index.sai.StorageAttachedIndex' WITH OPTIONS = {
'index_analyzer': '{
        "tokenizer" : {"name" : "standard"},
        "filters" : [{"name" : "porterstem"}]
}'};

```



In [ ]:
jobs_table = "CREATE TABLE IF NOT EXISTS " + os.environ.get("ASTRA_DB_KEYSPACE") + "." + os.environ.get("ASTRA_DB_TABLE_NAME") + " (id text PRIMARY KEY, title text, description text, company text, location text, url text, created_at timestamp, updated_at timestamp, deleted_at timestamp, vector list<float>);"
jobs_table_embedding_vector_idx = "CREATE CUSTON INDEX IF NOT EXISTS jobs_emb_vec_idx ON " + os.environ.get("ASTRA_DB_KEYSPACE") + "." + os.environ.get("ASTRA_DB_TABLE_NAME") + "(embedding_vector) USING 'StorageAttachedIndex';"
jobs_table_location_idx = "CREATE CUSTON INDEX IF NOT EXISTS jobs_loc_idx ON " + os.environ.get("ASTRA_DB_KEYSPACE") + "." + os.environ.get("ASTRA_DB_TABLE_NAME") + " (location) USING 'StorageAttachedIndex WITH OPTIONS = {'index_analyzer': '{'tokenizer': {'name':'standard'}, 'filters': [{'name': 'porterstem'}]}'};"
jobs_table_salary_idx = "CREATE CUSTOM INDEX IF NOT EXISTS jobs_sal_idx ON " + os.environ.get("ASTRA_DB_KEYSPACE") + "." + os.environ.get("ASTRA_DB_TABLE_NAME") + " (salary) USING 'StorageAttachedIndex' WITH OPTIONS = {'index_analyzer' = '{'tokenizer': {'name':'standard', 'filters':['{'name': 'porterstem'}]}'};"

## Configure AstraDB connection

In [2]:

cluster = Cluster(
    cloud={
        "secure_connect_bundle": ASTRA_DB_SECURE_BUNDLE_PATH,
    },
    auth_provider=PlainTextAuthProvider(
        "token", os.environ.get("ASTRA_API_KEY")
    ),
    protocol_version=ProtocolVersion.V4,
    execution_profiles={EXEC_PROFILE_DEFAULT: profile},
)

session = cluster.connect()
session.execute("USE " + os.environ.get("ASTRA_DB_KEYSPACE") + ";")
session.execute(jobs_table)
session.execute(jobs_table_embedding_vector_idx)
session.execute(jobs_table_location_idx)
session.execute(jobs_table_salary_idx)

ERROR:cassandra.connection:Closing connection <AsyncoreConnection(137240490095040) 4a68c548-f1bd-4264-8934-bf4e8d2f85fe-us-east1.db.astra.datastax.com:29042:05d703a1-936c-48cc-9ed8-c195f51b1c07> due to protocol error: Error from server: code=000a [Protocol error] message="Beta version of the protocol used (5/v5-beta), but USE_BETA flag is unset"


## Defining function for embedding texts

In [6]:
import openai
openai.api_key = os.environ.get("OPENAI_API_KEY")
def generate_embedding(text):
    model = "text-embedding-ada-002"
    response = openai.Embedding.create(model=model, input=text)
    return response.data[0]['embedding']

## Loading a CSV file into Astra vector database after creating embeddings for `job_description`

In [ ]:
import csv
from cassandra.query import SimpleStatement
from cassandra.concurrent import execute_concurrent_with_args
count = 0
# Inser query for the prepared statement
inser_embedding_stmt = f"""INSERT INTO """ + os.environ.get("ASTRA_DB_KEYSPACE") + """.""" + os.environ.get("ASTRA_DB_TABLE_NAME") + """ (job_id, job_title, skills, salary, location, embedding_vector) VALUES (?, ?, ?, ?, ?, ? )"""
prepared = session.prepare(inser_embedding_stmt)

# Input the CSV file for job listing
input_csv_file = 'jobs.csv'
print('Please upload your jobs.csv file')
jobs_sample_data_uploaded = files.upload()

# Open the input CSV file for reading
try:
    with open(input_csv_file, 'r', newline='') as csvfile:
        csvreader = csv.reader(csvfile, delimiter=';')
        next(csvreader)

        for row in csvreader:
            count += 1
            job_title     = ' '.join(row[0:1])
            skills        = ' '.join(row[2:3])
            salary        = ' '.join(row[3:4])
            location      = ' '.join(row[4:5])
            combined_text = ' '.join(row[1:2])
            print(count, row[0:1], row[2:3], row[3:4], row[4:5])
            embedding_res = generate_embedding(combined_text)
            #print((f"""INSERT INTO {ASTRA_DB_KEYSPACE}.{ASTRA_DB_TABLE_NAME} (job_id,job_title,skills,salary,location,embedding_vector ) VALUES (%s, %s, %s,%s, %s, %s )"""))
            #query = SimpleStatement(f"""INSERT INTO {ASTRA_DB_KEYSPACE}.{ASTRA_DB_TABLE_NAME} (job_id,job_title,skills,salary,location,embedding_vector  ) VALUES (%s, %s, %s,%s, %s, %s )""")
            session.execute(prepared, (str(count), job_title, skills, salary, location, embedding_res))

except FileNotFoundError:
    print(f"File '{input_csv_file}' not found.")
except Exception as e:
    print(f"An error occurred: {str(e)}")

1 ['Software Engineer'] ['Java, Python, JavaScript, SQL, Agile'] ['Salary: $90,000 - $120,000 per year'] ['Location: San Francisco, CA']
INSERT INTO vector.jobs (job_id,job_title,skills,salary,location,embedding_vector ) VALUES (%s, %s, %s,%s, %s, %s )
2 ['Data Analyst'] ['SQL, Excel, Data Visualization, Statistics'] ['Salary: $70,000 - $90,000 per year'] ['Location: New York, NY']
INSERT INTO vector.jobs (job_id,job_title,skills,salary,location,embedding_vector ) VALUES (%s, %s, %s,%s, %s, %s )
3 ['Network Administrator'] ['Cisco, VPN, Network Security, Troubleshooting'] ['Salary: $75,000 - $100,000 per year'] ['Location: Los Angeles, CA']
INSERT INTO vector.jobs (job_id,job_title,skills,salary,location,embedding_vector ) VALUES (%s, %s, %s,%s, %s, %s )
4 ['UX Designer'] ['User Research, Wireframing, Prototyping, UI Design'] ['Salary: $80,000 - $110,000 per year'] ['Location: Austin, TX']
INSERT INTO vector.jobs (job_id,job_title,skills,salary,location,embedding_vector ) VALUES (%s, %

## Defining a function that will be used for evaluating the results

It is using gpt-4 to evaluate the results after having the top similar results from Astra DB

In [7]:
def get_completion_from_messages(messages, model="gpt-4", temperature=0):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

## Testing with sample CVs to find the best possible job posting for a given CV



*   For each sample CV in the file, combine `text search` to look for `location` and get the best results for similarity between job_posting and CV in Astra DB
*   If `similarity_search score` is bigger than a threshold, send the results to LLM to get a new scoring to confirm that we have an ideal job for the candidate.

We have only one Marketin Manager position is in Chicago so there was no result for Bob Smith after `text search` for `Location`.


In [8]:
import csv
from cassandra.query import SimpleStatement
count = 0
# Inser query for the prepared statement
select_embedding_stmt = "SELECT job_id, job_title, skills, salary, location, embedding_vector, similarity_cosine(embedding_vector, %s) as score  FROM " + os.environ.get("ASTRA_DB_KEYSPACE") + "." + os.environ.get("ASTRA_DB_TABLE_NAME") + " where location: %s ORDER BY embedding_vector ANN OF %s LIMIT 3"
prepared_emb = session.prepare(inser_embedding_stmt)

input_csv_file = 'test_cvs.csv'
print('Please upload your test_cvs.csv file')
cv_sample_data_uploaded = files.upload()

try:
    with open(input_csv_file, 'r', newline='') as csvfile:
        csvreader = csv.reader(csvfile,delimiter=';' )
        next(csvreader)

        for row in csvreader:
            count += 1
            name = ' '.join(row[0:1])
            job_title=' '.join(row[1:2])
            cv = ' '.join(row[2:3])
            location = ' '.join(row[3:4])
            salary = ' '.join(row[4:5])
            print("Search similarity for this CV:",cv,"\n")
            embedding_res = generate_embedding(cv)
            #query = SimpleStatement(f"""SELECT job_id,job_title,skills,salary,location,embedding_vector,similarity_cosine(embedding_vector, {embedding_res}) as score  FROM """ + os.environ.get("ASTRA_DB_KEYSPACE") + """.""" + os.environ.get("ASTRA_DB_TABLE_NAME") + """ where location: '{location}' ORDER BY embedding_vector ANN OF {embedding_res} LIMIT 3""")
            print(name, location)
            #res = session.execute(query)
            res = session.execute(prepared_emb, embedding_res, location, embedding_res)
            for row in res:
              res_job = row.job_title
              id = row.job_id
              if row.score > 0.91:
                  print('Result: Score',row.score,' Job_id ',id,' ',res_job,"\n")
                  messages =  [
                  {'role':'system', 'content':'You are a chatbot for giving scores for the result of a job posting and CV comparison that are sent in [].You will help eliminating the candidates that doesnt fit the role by ranking them close to 1.'},
                  {'role':'system', 'content':'You need to give a score between 1 and 10. If it is a good candidate for the job , you can give 10. '},
                  {'role':'system', 'content': "You should give a detailed explanation how you decide the ranking and give the ranking result as a number at the end."},
                  {'role':'user', 'content':f'[{cv}],[{res_job}]' } ]
                  response = get_completion_from_messages(messages, temperature=0)
                  print(response)
            if count ==5:
             break
            print("#########################################################")

except FileNotFoundError:
    print(f"File '{input_csv_file}' not found.")
except Exception as e:
    print(f"An error occurred: {str(e)}")

Search similarity for this CV: I am an experienced Software Engineer with expertise in Java, Python, and JavaScript. I have a strong background in software development and have worked on various projects, including web applications and backend systems. My skills include database design, API development, and problem-solving. I am passionate about writing clean and maintainable code and enjoy working in agile teams to deliver high-quality software solutions. 

John Doe San Francisco
Result: Score 0.9292247295379639  Job_id  1   Software Engineer 

Based on the job posting for a Software Engineer and the CV provided, the candidate seems to be a strong match. The candidate has experience in software development and has worked with Java, Python, and JavaScript, which are commonly used languages in software engineering. They also have experience with database design and API development, which are valuable skills for a Software Engineer. The candidate's passion for writing clean and maintaina